In [1]:
# all the imports
!pip install transformers
!pip install pandas
!pip install sklearn
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install tensorboard tensorboardx

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [2]:
!pip install pyarrow

import pandas as pd

# feather_path = '/content/drive/Othercomputers/MacBookPro/chatbot/0926_학습용_일상대화_감성대화.feather'
feather_path = '/home/hyeokiki/Desktop/ko_dialogpt/1012_학습용_일상대화_감성대화_대화체관리가이드.feather'
f = pd.read_feather(feather_path)
f

,data
0,"['요즘 동물들을 유기 하는 애들도 참 문제야 ㅠㅠ', '동물들을 유기 하는 사람이..."
1,"['강릉에 어떤 호텔에서 오징어 게임 한데', '오징어게임? 설마 죽는 거 아이제..."
2,"['이렇게 너와 내가 국제 관계를 트는구나', '그래 해외에 친구가 있는 것도 나쁘..."
3,"['너는 전철 자주 타고 다니는 편이야?', '예전엔 많이 타고 다녔는데 요즘 통 ..."
4,"['귀멸의 칼날 봤니?', '응 당연 봤지', '안 볼 수가 없지', '키키 극장판..."
...,...
128650,"['스파티필룸 왈리시(스파티필룸)', '흰색의 불염포와 짙은 녹색의 윤기 있는 잎이..."
128651,"['싱고니움', '싱고니움은 대표적인 잎보기 식물로 연두색, 흰색, 분홍색 등의 잎..."
128652,"['마삭줄 / 오색마삭줄 / 황금마삭줄', '우리나라 남부지방에서 주로 자생하며 상..."
128653,"['튤립', '터키가 원산지인 튤립은 가을에 심는 구근식물로 실내에서는 겨울부터 봄..."


In [5]:
# new_df = pd.DataFrame(columns=['data'])

# ko_list = []
# for i in range(0, len(f)):
#   s = f.iloc[i]
#   s = s.apply(lambda x: eval(x))
#   for j in range(len(s[0])):
#     s[0][j] = s[0][j].strip('\n')
#   new_df.loc[len(new_df)] = str(s[0])
# new_df.to_feather('test.feather')



In [6]:
# new_df.to_feather('/content/drive/Othercomputers/MacBookPro/chatbot/일상대화.feather')

In [7]:
f.apply(lambda x : len(eval(x[0])), axis = 1).describe()

count    128655.000000
mean         12.953923
std           7.062172
min           1.000000
25%           6.000000
50%          12.000000
75%          17.000000
max         108.000000
dtype: float64

In [14]:
import pprint


# WINDOW = 7
WINDOW = 17

train_data = []

def gen_context(data):
  # data = eval(f.iloc[0][0])
  
  for i in range(len(data)) :
    # print(data[i])
    context = data[i + 1 - WINDOW: i + 1][::-1]
    if len(context) != WINDOW : continue
    # print(context)
    train_data.append(context)

gen_context(eval(f.iloc[0][0]))
train_data

[['코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...',
  '거기다가 변이 바이러스까지 겹쳐서 걱정이다...',
  '그게 난 정말 의문이야 ㅠㅠ',
  '백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...',
  '그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?',
  '휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직',
  '코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라',
  '며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아',
  '아무래도 가장 큰 건 코로나 아닐까?',
  '요즘 사회적 이슈들은 뭐가 있을까?',
  '다 벌 받을 거야 나쁜 사람들 ㅠ',
  '**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데',
  '고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워',
  '우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ',
  '그치 동물들도 요즘 사회적 이슈고 요즘 또 이슈화 있는 사건이 많더라고',
  '동물들을 유기 하는 사람이 아직도 있어?',
  '요즘 동물들을 유기 하는 애들도 참 문제야 ㅠㅠ'],
 ['다음 대통령은 꼭 코로나를 잡진 못하더라도 완만히 넘길 수 있는 대통령이 나왔으면 좋겠어!',
  '코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...',
  '거기다가 변이 바이러스까지 겹쳐서 걱정이다...',
  '그게 난 정말 의문이야 ㅠㅠ',
  '백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...',
  '그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?',
  '휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직',
  '코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라',
  '며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아',
  '

In [15]:
f.apply(lambda x : gen_context(eval(x[0])), axis = 1)

0         None
1         None
2         None
3         None
4         None
          ... 
128650    None
128651    None
128652    None
128653    None
128654    None
Length: 128655, dtype: object

In [16]:
columns = ['response', 'context'] 
columns = columns + ['context/' + str(i) for i in range(WINDOW - 2)]

df = pd.DataFrame.from_records(train_data, columns=columns)
df

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8,context/9,context/10,context/11,context/12,context/13,context/14
0,코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...,거기다가 변이 바이러스까지 겹쳐서 걱정이다...,그게 난 정말 의문이야 ㅠㅠ,백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...,그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?,휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직,코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라,며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아,아무래도 가장 큰 건 코로나 아닐까?,요즘 사회적 이슈들은 뭐가 있을까?,다 벌 받을 거야 나쁜 사람들 ㅠ,**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데,고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워,우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ,그치 동물들도 요즘 사회적 이슈고 요즘 또 이슈화 있는 사건이 많더라고,동물들을 유기 하는 사람이 아직도 있어?,요즘 동물들을 유기 하는 애들도 참 문제야 ㅠㅠ
1,다음 대통령은 꼭 코로나를 잡진 못하더라도 완만히 넘길 수 있는 대통령이 나왔으면 ...,코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...,거기다가 변이 바이러스까지 겹쳐서 걱정이다...,그게 난 정말 의문이야 ㅠㅠ,백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...,그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?,휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직,코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라,며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아,아무래도 가장 큰 건 코로나 아닐까?,요즘 사회적 이슈들은 뭐가 있을까?,다 벌 받을 거야 나쁜 사람들 ㅠ,**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데,고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워,우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ,그치 동물들도 요즘 사회적 이슈고 요즘 또 이슈화 있는 사건이 많더라고,동물들을 유기 하는 사람이 아직도 있어?
2,다음 대통령은 과연 누가 될까? 궁금하다,다음 대통령은 꼭 코로나를 잡진 못하더라도 완만히 넘길 수 있는 대통령이 나왔으면 ...,코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...,거기다가 변이 바이러스까지 겹쳐서 걱정이다...,그게 난 정말 의문이야 ㅠㅠ,백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...,그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?,휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직,코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라,며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아,아무래도 가장 큰 건 코로나 아닐까?,요즘 사회적 이슈들은 뭐가 있을까?,다 벌 받을 거야 나쁜 사람들 ㅠ,**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데,고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워,우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ,그치 동물들도 요즘 사회적 이슈고 요즘 또 이슈화 있는 사건이 많더라고
3,내가 생각했을 때 대통령이 제일 중요한 인물이기 때문에 좋은 사람이 되기를 바래야지 ㅠㅠ,다음 대통령은 과연 누가 될까? 궁금하다,다음 대통령은 꼭 코로나를 잡진 못하더라도 완만히 넘길 수 있는 대통령이 나왔으면 ...,코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...,거기다가 변이 바이러스까지 겹쳐서 걱정이다...,그게 난 정말 의문이야 ㅠㅠ,백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...,그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?,휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직,코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라,며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아,아무래도 가장 큰 건 코로나 아닐까?,요즘 사회적 이슈들은 뭐가 있을까?,다 벌 받을 거야 나쁜 사람들 ㅠ,**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데,고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워,우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ
4,그치 우리한테는 직접적으로까진 아니어도 분명 영향을 줄 거야 대통령이라는 사람은,내가 생각했을 때 대통령이 제일 중요한 인물이기 때문에 좋은 사람이 되기를 바래야지 ㅠㅠ,다음 대통령은 과연 누가 될까? 궁금하다,다음 대통령은 꼭 코로나를 잡진 못하더라도 완만히 넘길 수 있는 대통령이 나왔으면 ...,코로나만 어떻게 해결된다면 사회적 이슈가 잦아질텐데 말이야...,거기다가 변이 바이러스까지 겹쳐서 걱정이다...,그게 난 정말 의문이야 ㅠㅠ,백신이 아직까지는 큰 효과를 보지는 못하는 거 같아...,그런데 백신을 그렇게 많은 사람들이 맞고 있는데 왜 더 늘어날까?,휴 추석이 끝나고 1주일 동안은 엄청 늘어날 거라고 하더니 그래도 생각보단 괜찮네 아직,코로나가 제일 심각한 문제긴 하지 우리나라 뿐만 아니라 전 세계적으로 문제가 있는 거라,며칠 전에 3200명이란 어마어마한 확진자가 발생했잖아,아무래도 가장 큰 건 코로나 아닐까?,요즘 사회적 이슈들은 뭐가 있을까?,다 벌 받을 거야 나쁜 사람들 ㅠ,**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데,고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217988,난 기대고 싶어요 키키,오빠도 좀차이가났어야되는데ㅡㅡ,키키 소리치고 키키 나도 뭐 생각안해봐서 키키,지는 오빠가 좋아요,난 내 보다 어린애가 내 이름 부르면 기분 나쁠 거 같아요 키키,헐 키키 니는 오빠가좋으나?,내 친구 신랑들은 한 열 살 많은 어르신 같음 키키,키키 결혼하면 다똑같음 키키,근데 뭔가 연하 하면은 되게 좀 사랑스럽고 키키 상큼하고 키키 이런 이미진데~,키키 그자? 나는 그냥 친구가 좋았는듯,근데 연하 느낌1도 안남 키키,내 친구 두명은 연하랑 결혼했거든요~,키키 미안 연하는 좀 안 맞는 듯 키키,나도 연하는 안 만나 봐서 궁금해서요 언니 키키,오빠도 1살 오빠 니는?,아니 난 동갑,언니 연하 남친 만나 봤어요?
217989,누가 나한테 기대는 거 말고 키키,난 기대고 싶어요 키키,오빠도 좀차이가났어야되는데ㅡㅡ,키키 소리치고 키키 나도 뭐 생각안해봐서 키키,지는 오빠가 좋아요,난 내 보다 어린애가 내 이름 부르면 기분 나쁠 거 같아요 키키,헐 키키 니는 오빠가좋으나?,내 친구 신랑들은 한 열 살 많은 어르신 같음 키키,키키 결혼하면 다똑같음 키키,근데 뭔가 연하 하면은 되게 좀 사랑스럽고 키키 상큼하고 키키 이런 이미진데~,키키 그자? 나는 그냥 친구가 좋았는듯,근데 연하 느낌1도 안남 키키,내 친구 두명은 연하랑 결혼했거든요~,키키 미안 연하는 좀 안 맞는 듯 키키,나도 연하는 안 만나 봐서 궁금해서요 언니 키키,오빠도 1살 오빠 니는?,아니 난 동갑
217990,키키 그래 난 계속 독립적이다,누가 나한테 기대는 거 말고 키키,난 기대고 싶어요 키키,오빠도 좀차이가났어야되는데ㅡㅡ,키키 소리치고 키키 나도 뭐 생각안해봐서 키키,지는 오빠가 좋아요,난 내 보다 어린애가 내 이름 부르면 기분 나쁠 거 같아요 키키,헐 키키 니는 오빠가좋으나?,내 친구 신랑들은 한 열 살 많은 어르신 같음 키키,키키 결혼하면 다똑같음 키키,

In [17]:
trn_df, val_df = train_test_split(df, test_size=0.1)
trn_df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8,context/9,context/10,context/11,context/12,context/13,context/14
72564,몰라 키키,진짜? 키키 왜 만들어 놓은 거야 키키,아? 키키 집에 설치하느라 힘들었나 보네 키키,근데 설치해놓고 지쳤나 안 쓰더라 키키,그러게ㅜㅜ 맨날 친구 집에 놀러 가는 거 아니야? 키키,친구 부럽다 ㅠㅠ,헐... 완전 내 워너비 ㅠㅠ,내 친구는 주택에 사는데 옥상에다 아예 캠핑장을 만들어놨어 키키,거의 꿈의 이야기다 ㅜㅜ,그럼 진짜 좋지 키키,집에 바베큐장 딱 만들어 놓고 주말마다 바베큐 파티하고 ㅠㅠ,그러게 갑자기 현타가 오네 키키,뭐야 둘 다 가능성이 희박해 ㅜㅜ,그치... 주택 살거나 건물주라 옥상에 바베큐장을 만들거나...키키,맞아 키키 주택 사는 거 아니면 힘들지 않나?,집에 바베큐장 있으면 대박이지 키키,집에 바베큐장 있었으면 좋겠다 ㅠㅠ
132048,다들 무조건 아이스구나 키키,따뜻한 커피는 왜 맛이 없니...?,물도 같이 꼭 먹어야 해,힘이 안나 커피 안 마시면,물을 많이 마셔야 한데 진짜...!,나도 무조건 아이스 먹어,물도 많이 마셔줘야 한다구,얼죽아지 키키 물 마셔야 해,아니면 기분이 안 좋아 키키,물도 많이 마셔줘야 해 ㅠㅠ,나도 당근 아이스지 하하,하루 한잔 커피 필수지 ㅠㅠ,얼죽아니 다들? 맞아 한잔 필수,하루 한잔 필수야,나는 물 먹고 있어 지금,얼죽아야 겨울에도 아이스 하하,난 커피 마시는 중... 하하
199219,사직 구장 새로 짓는다고 하더라,역시 기업이 운영해야 된다 스포츠는,나는 모르는 소리야 키키,스타벅스 생기고 개쩜...,랜더스 세련되고 현대적이야,코로나라서 잘 안 팔리나?,키키 랜더스 요즘 난리잖아 ㅠ,직관 자주 온대,용진이형 하하 보고싶당,지금 할인이면 바로 고고해야지,코로나 어쩌고 해서 50%!,용진이형 랜더스 응원하고 싶음 야구,나도 야구장 안 간지 오래됨 ㅜㅜ,야구장 지금 할인한데,지금이라도 농구 하면 키 크려나,윽 요즘은 비시즌인가?,야구장 가고 싶다
140467,네 지금 생각해보면 그랬던 것 같아요 키키,***님 스트레스 해소용? 키키,한마디로 감정 쓰레기통이었다는,키키,저도 참 못된게 맘 힘들고 그럴때 전화해서 꼬장부렸어요,제가 서울 살때 헤어졌는데,정 다 떨어지고 헤어진거라,전 헤어지고는 쿨하게 떠나는데...,남자도 참...,저 찾으면서 키키,키키,대학 선배들 동기들 다 친하니까 모임같은것도 많았는데 그때마다 울었대요,욕하고 그랬어요? 키키,엄청 했어요,술먹고 꼬장 키키 여자가 하는건 처음봤어요 키키,술먹고 전화해서 꼬장부리구요 네네,다시 사귀어줄거처럼? 키키
196581,그림은 귀엽고 스토리도 괜찮던데... 다음에 시간 좀 많을 때 한번 다시 도전해봐야겠어.,맞아...시작한지 얼마안됐는데 접어야할까봐.,스토리가 좀 재미있어서 하다가 너무 어렵고 화가나서 못하겠어.,나는 원래 못하는 편이긴 한데...,"내가 진짜 게임 못하는 편이 아닌데, 블소는 나랑 안맞는지 죽고 또 죽고 또또 죽더라.",ㅋ엄청 욕먹었잖아,나는 내가 앞장서서 몬스터랑 싸우고 그래야하는데 그런건지도 모르고 맨 뒤에서 혼자 ...,그거 너무 과하게 어려워 ㅠㅠ,나는 그거 파티사냥 해볼라다가 내가 계속 죽어서 못하겠어서 접었어... ㅠㅠ,그래서 잘 못해 키키,응 어려워 ㅋ,인던에서 무슨 페이즈 맞춰서 공격하는것도 엄청 어렵던데,막 하늘 나는법도 어렵고...,야 그거 잠깐 해봤는데 어렵던데;,오 그거 재밌나?,나 요새 친구랑 블레이드 앤 소울해 ㅋ,새로나온것도 없고 모르겠따


In [18]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [19]:
# create dataset suitable for our model
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        try :
          return torch.tensor(self.examples[item], dtype=torch.long)
        except:
          print(f'item error : {item}, {self.examples[item]}')


In [20]:
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch

# tokenizer = AutoTokenizer.from_pretrained("/content/drive/Othercomputers/MacBookPro/chatbot/ko_dialogpt/pleon-tokenizer")
# model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-medium")

from transformers import BertTokenizerFast, GPT2LMHeadModel
# tokenizer_gpt3 = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
# input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token
# model_gpt3 = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

In [21]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [22]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = '/home/hyeokiki/Desktop/ko_dialogpt'
        self.model_type = 'gpt2'
        # self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.model_name_or_path = 'beomi/kykim-gpt3-kor-small_based_on_gpt2'
        # self.config_name = 'microsoft/DialoGPT-small'
        self.config_name = 'beomi/kykim-gpt3-kor-small_based_on_gpt2'
        # self.tokenizer_name = '/content/drive/Othercomputers/MacBookPro/chatbot/ko_dialogpt/pleon-tokenizer'
        self.tokenizer_name = 'beomi/kykim-gpt3-kor-small_based_on_gpt2'
        # self.cache_dir = '/content/drive/Othercomputers/MacBookPro/chatbot/dialogpt_small/cached'
        self.cache_dir = 'cached'

        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 32
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 100000 #원래 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = True
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [23]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)


    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [24]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            shold_continue = False
            # raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    # config = AutoConfig.from_pretrained(args.config_name)#, cache_dir=args.cache_dir)
    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    
    print(args.tokenizer_name)
    #tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)#, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)

    # tokenizer = BertTokenizerFast.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    tokenizer.add_special_tokens({"eos_token":"[SEP]",
                               "bos_token":"[SEP]",})
    
    # model = AutoModelWithLMHead.from_pretrained(
    #     args.model_name_or_path,
    #     from_tf=False,
    #     config=config,
    # )
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )

    # model = GPT2LMHeadModel.from_pretrained(
    #     args.model_name_or_path,
    #     from_tf=False,
    #     config=config,
    #     cache_dir=args.cache_dir,
    # )
    
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        # model = GPT2LMHeadModel.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        tokenizer.add_special_tokens({"eos_token":"[SEP]",
                               "bos_token":"[SEP]",})
        # tokenizer = BertTokenizerFast.from_pretrained(args.output_dir)

        # tokenizer = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            # model = GPT2LMHeadModel.from_pretrained(checkpoint)
            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)
             
    return results

In [27]:
main(trn_df, val_df)


11/03/2022 16:53:59 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False


beomi/kykim-gpt3-kor-small_based_on_gpt2


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

11/03/2022 16:54:50 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7fec1ece0040>
11/03/2022 16:54:50 - INFO - __main__ -   Creating features from dataset file at cached
11/03/2022 16:56:16 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_510
11/03/2022 16:56:16 - INFO - __main__ -   ***** Running training *****
11/03/2022 16:56:16 - INFO - __main__ -     Num examples = 196193
11/03/2022 16:56:16 - INFO - __main__ -     Num Epochs = 32
11/03/2022 16:56:16 - INFO - __main__ -     Instantaneous batch size per GPU = 4
11/03/2022 16:56:16 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
11/03/2022 16:56:16 - INFO - __main__ -     Gradient Accumulation steps = 1
11/03/2022 16:56:16 - INFO - __main__ -     Total optimization steps = 1569536


Epoch:   0%|          | 0/32 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/49048 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49048 [00:00<?, ?it/s]

11/03/2022 23:10:07 - INFO - __main__ -   Saving model checkpoint to /home/hyeokiki/Desktop/ko_dialogpt/checkpoint-300000
11/03/2022 23:10:08 - INFO - __main__ -   Saving optimizer and scheduler states to /home/hyeokiki/Desktop/ko_dialogpt/checkpoint-300000


Iteration:   0%|          | 0/49048 [00:00<?, ?it/s]

KeyboardInterrupt: 

beomi/kykim-gpt3-kor-small_based_on_gpt2'

In [ ]:
tokenizer.add_special_tokens({"eos_token":"[SEP]",
                               "bos_token":"[SEP]",})

In [ ]:
tokenizer1.bos_token_id

In [ ]:
temp_df = trn_df.apply(lambda x: tokenizer.encode(' '.join(list(x))), axis=1)


In [ ]:
temp_df[temp_df.apply(lambda x: len(x)) ==0]

In [ ]:
construct_conv1(trn_df.loc[295357], tokenizer=tokenizer)

In [ ]:
trn_df.loc[295357]

In [ ]:
def construct_conv1(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([[x] + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

In [ ]:
for i, row in trn_df.iterrows():
  if i != 295357:
    continue
  print(i, row)

In [ ]:
print(tokenizer.all_special_ids)